### Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user adoption .

### We  suggest  spending  1-2  hours  on  this,  but  you're  welcome  to  spend  more  or  less. Please  send  us  a  brief  writeup  of  your  findings  (the  more  concise,  the  better   no  more than  one  page),  along  with  any  summary  tables,  graphs,  code,  or  queries  that  can  help us  understand  your  approach.  Please  note  any  factors  you  considered  or  investigation you  did,  even  if  they  did  not  pan  out.  Feel  free  to  identify  any  further  research  or  data you  think  would  be  valuable.

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
import os
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
os.getcwd()

'C:\\Users\\jonat\\Desktop\\Career\\Springboard\\relax_challenge'

In [76]:
#specific encoding had to specified in this instance to open the csv correctly
users = pd.read_csv('takehome_users.csv',encoding = 'latin')
user_engagement = pd.read_csv('takehome_user_engagement.csv')

In [4]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [6]:
user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [7]:
user_engagement.sort_values(['user_id','time_stamp'])
user_engagement

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [8]:
filtered_engagement = user_engagement[user_engagement.groupby('user_id').transform('count')>2]
user_engagement.groupby('user_id').transform('count')>2

,time_stamp,visited
0,False,False
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
207912,False,False
207913,False,False
207914,False,False
207915,False,False


The process of figuring out a strict 7 day window where 3 or more logins are present is becoming too tenuous... I will just use weekly resampled groupings instead

In [9]:
user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp'])
users['creation_time'] = pd.to_datetime(users['creation_time'])

In [10]:
user_engagement = user_engagement.set_index('time_stamp')

In [11]:
weekly_engagement = user_engagement.groupby('user_id').resample('W').sum()

In [12]:
adopted_users = weekly_engagement[weekly_engagement['visited']>2]

In [13]:
adopted_users

user_id  visited
user_id time_stamp                  
2       2014-02-09        6        3
10      2013-03-03       30        3
        2013-04-14       30        3
        2013-04-28       30        3
        2013-05-05       40        4
...                     ...      ...
11988   2014-04-06    35964        3
        2014-05-04    47952        4
        2014-05-18    35964        3
        2014-05-25    59940        5
        2014-06-01    35964        3

[33859 rows x 2 columns]

In [14]:
adoptees = []
for i in range(len(adopted_users.index)):
    adoptees.append(adopted_users.index[i][0])
adoptees = set(adoptees)

adoption_percent = int(len(adoptees))/len(set(list(user_engagement['user_id'])))
adoption_percent*100

16.377649325626205

Only about 16.4% of the users historically adopted the product

In [15]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [16]:
#add a new column for adoption, set as false by default since most records will be false
users['adoption'] = False

#run for loop to update adopted users
for a in adoptees:
    users.loc[a,'adoption'] = True

In [17]:
users[users['adoption']==True]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adoption
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,True
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,0,0,69,NaN,True
20,21,2013-01-22 12:27:42,Hunter Alfie,AlfieHunter@gmail.com,PERSONAL_PROJECTS,1.358858e+09,0,0,54,NaN,True
33,34,2014-02-03 20:55:18,Holm Sara,SaraMHolm@hotmail.com,ORG_INVITE,NaN,0,0,33,3721.0,True
42,43,2012-12-31 07:13:17,Gomes Beatrice,vgqmpljq@yyyxt.com,GUEST_INVITE,1.366010e+09,0,0,63,149.0,True
...,...,...,...,...,...,...,...,...,...,...,...
11965,11966,2013-01-01 02:28:35,Ferreira Luana,LuanaAzevedoFerreira@gmail.com,PERSONAL_PROJECTS,NaN,0,0,39,NaN,True
11967,11968,2013-04-01 05:00:18,Thorsen Marcus,MarcusMThorsen@gmail.com,SIGNUP_GOOGLE_AUTH,1.365224e+09,1,1,273,NaN,True
11969,11970,2013-10-19 09:51:09,Bartlett Brooke,BrookeBartlett@gmail.com,SIGNUP,NaN,0,0,126,NaN,True
11975,11976,2013-12-25 22:01:41,Kohl Leah,LeahKohl@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,248,NaN,True


## Feature Engineering

### Before modeling, let's see if we can juice this rock more to get more useful data points ut of our existing columns

In [18]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  object        
 5   last_session_creation_time  8823 non-null   float64       
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          6417 non-null   float64       
 10  adoption                    12000 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(4), o

**we should change some of these datatypes**

In [19]:
users['opted_in_to_mailing_list'] = users['opted_in_to_mailing_list'].astype('bool')
users['enabled_for_marketing_drip'] = users['enabled_for_marketing_drip'].astype('bool')

**Email Clients may be more useful than individual email address**

In [20]:
#this will split up the email and append the email_client list with the domains (everything after the @))
email_client = []
for e in users['email']:
    email_client.append(str(e).split('@')[-1])

In [21]:
#we see that there are only 6 primary email clients, the rest can just be categorized as other
all_domains = set(pd.Series(email_client))
primary_domains = set(['gmail.com','yahoo.com','jourrapide.com','cuvox.de','gustr.com','hotmail.com'])
all_domains -=primary_domains

In [22]:
users['email_client'] = pd.Series(email_client)

In [23]:
for i in range(len(email_client)):
    if email_client[i] in (all_domains):
        users.loc[i,'email_client'] = None

In [24]:
users['email_client'].value_counts()

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
gustr.com         1179
hotmail.com       1165
Name: email_client, dtype: int64

**Much Better!**

In [25]:
users['creation_source'].value_counts()

ORG_INVITE            4254
GUEST_INVITE          2163
PERSONAL_PROJECTS     2111
SIGNUP                2087
SIGNUP_GOOGLE_AUTH    1385
Name: creation_source, dtype: int64

**this is looking pretty clean!**

In [26]:
len(users['creation_source'])

12000

In [27]:
users['is_invited'] = users['invited_by_user_id'].isnull()==False

In [28]:
users['is_invited'].value_counts()

True     6417
False    5583
Name: is_invited, dtype: int64

In [29]:
users[users['invited_by_user_id'].isnull()]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adoption,email_client,is_invited
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,False,True,37,NaN,False,jourrapide.com,False
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,True,True,74,NaN,False,yahoo.com,False
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,False,False,302,NaN,False,hotmail.com,False
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,False,False,69,NaN,True,gustr.com,False
13,14,2012-10-11 16:14:33,Rivera Bret,BretKRivera@gmail.com,SIGNUP,1.350058e+09,False,False,0,NaN,False,gmail.com,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11992,11993,2013-03-28 23:24:21,Townsend Isabel,IsabelTownsend@cuvox.de,PERSONAL_PROJECTS,NaN,False,False,281,NaN,False,cuvox.de,False
11994,11995,2013-10-21 13:33:17,Werfel Sebastian,SebastianWerfel@gmail.com,PERSONAL_PROJECTS,1.382449e+09,False,False,0,NaN,False,gmail.com,False
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,False,False,200,NaN,False,gmail.com,False
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,False,False,6,NaN,False,None,False


### Dummy variables summary: 
* email client domain - done
* creation_source - make into dummy columns
* invited_by_user - converted into is_invited
* email opt in - keep as bool
* enabled for drip keep as bool
* org_id - make into dummy ... maybe omit entirely because there are so many values 

In [30]:
users = pd.get_dummies(users,columns=['creation_source','email_client'])

In [31]:
users.head()

,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adoption,...,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,email_client_cuvox.de,email_client_gmail.com,email_client_gustr.com,email_client_hotmail.com,email_client_jourrapide.com,email_client_yahoo.com
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,1.398139e+09,True,False,11,10803.0,False,...,0,0,0,0,0,0,0,0,0,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,1.396238e+09,False,False,1,316.0,False,...,1,0,0,0,0,0,1,0,0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,1.363735e+09,False,False,94,1525.0,True,...,1,0,0,0,0,0,1,0,0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,1.369210e+09,False,False,1,5151.0,False,...,0,0,0,0,0,0,0,0,0,1
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,1.358850e+09,False,False,193,5240.0,False,...,0,0,0,0,0,0,0,0,0,1


In [32]:
regressand = np.array(users['adoption'].astype(int))
regressors = users.drop(['adoption','creation_time','object_id','name','email','last_session_creation_time','org_id','invited_by_user_id'],axis=1)

In [33]:
regressors['opted_in_to_mailing_list'] = regressors['opted_in_to_mailing_list'].astype(int)
regressors['enabled_for_marketing_drip'] = regressors['enabled_for_marketing_drip'].astype(int)
regressors['is_invited'] = regressors['is_invited'].astype(int)

## Modeling

### I will be using a logistic regression model for the sake of simplicity and interpretability

In [34]:
X_train,X_test,y_train,y_test = train_test_split(regressors,regressand,random_state=6,test_size=0.3)

In [35]:
model1 = linear_model.LogisticRegression(solver='liblinear')
model1.fit(X_train,y_train)
y_pred = model1.predict(X_test)

In [36]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      3174
           1       0.00      0.00      0.00       426

    accuracy                           0.88      3600
   macro avg       0.44      0.50      0.47      3600
weighted avg       0.78      0.88      0.83      3600



c:\users\jonat\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
model1.score(X_test,y_test)

0.8816666666666667

In [38]:
cm = metrics.confusion_matrix(y_test,y_pred)
cm

array([[3174,    0],
       [ 426,    0]], dtype=int64)

Although the score is decent at 0.88, this is actually a terrible outcome because it failed to classify anything correct for the minority case (user adoption), at 0 Recall. This is especially bad because the adoption = 1 is a much smaller group and the poor recall rate really emphasizes how poorly the current model is at making correct predictions for future adoption. 

Lets shift gears and try something else as the minority case classification accuracy is more important in this instance for the problem at hand. 

## Trying random forest instead

In [39]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [59]:
clf = rfc(random_state=6,class_weight = 'balanced')
clf.fit(X_train,y_train)
y_pred_rfc = clf.predict(X_test)

In [60]:
training_score = clf.score(X_train,y_train)
training_score

0.539047619047619

In [61]:
testing_score = clf.score(X_test,y_test)
testing_score

0.5072222222222222

In [62]:
print(metrics.classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.89      0.51      0.64      3174
           1       0.12      0.52      0.20       426

    accuracy                           0.51      3600
   macro avg       0.50      0.51      0.42      3600
weighted avg       0.80      0.51      0.59      3600



This offers an improvement on recall which is a change for the better... Now lets take a look at the most important features that are driving these results...

In [72]:
key_features = pd.DataFrame({'RF Importance Coefficient':clf.feature_importances_},index=regressors.columns)
key_features.sort_values('RF Importance Coefficient',ascending=False)

,RF Importance Coefficient
opted_in_to_mailing_list,0.211650
enabled_for_marketing_drip,0.185777
email_client_gustr.com,0.084761
email_client_yahoo.com,0.063765
email_client_jourrapide.com,0.062432
email_client_hotmail.com,0.060805
email_client_cuvox.de,0.058430
email_client_gmail.com,0.058066
creation_source_SIGNUP,0.050354
creation_source_PERSONAL_PROJECTS,0.045713


### Analysis

**From the above results, the key indicators to whether customers adopt the product seem to be whether they're receiving marketing material through drip and being opted in on the mailing list. In addition, it seems that signing up with an email from gustr, jourrapide, hotmail, cuvox and gmail along with being invited to join are also a good indication of future adoption.**

**From these results, I think these are some strong recommendations**
* Make is to make a promotion campaign to incentivize current users inviting new users onto the platform
* Include promo codes for discounts to those from one of the 4 above email clients on the mailing list to encourage continued use
* Create some more content catering towards those using the platform for personal projects to attract continued visits